### ПОДКЛЮЧЕНИЕ ИСПОЛЬЗУЕМЫХ БИБЛИОТЕК

In [10]:
import sys

from PyQt5 import uic
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QMessageBox, qApp

from matplotlib.backends.backend_qt4agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

import matplotlib.pyplot as plt
import numpy as np
import numexpr as ne

from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from stl import mesh
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D

# РОДИТЕЛЬСКИЙ КЛАСС

In [11]:
class Window(QMainWindow):
    EXIT_CODE_REBOOT = -123
    def __init__(self, ui):
        super(Window,self).__init__()
        self.ui = uic.loadUi(ui, self)
        self.click()
    
    def SetWidget(self):
        self.figure = plt.Figure()
        self.canvas = FigureCanvas(self.figure)
        self.toolbar = NavigationToolbar(self.canvas, self)
        
        layout = QVBoxLayout()
        layout.addWidget(self.canvas)
        layout.addWidget(self.toolbar)
        self.w1.setLayout(layout)
        self.ax = self.figure.add_subplot(1,1,1)
    
    def SetWidget3D(self):
        self.figure2 = plt.Figure()
        self.canvas2 = FigureCanvas(self.figure2)
        self.toolbar2 = NavigationToolbar(self.canvas2, self)
        
        layout = QVBoxLayout()
        layout.addWidget(self.canvas2)
        layout.addWidget(self.toolbar2)
        self.w2.setLayout(layout)
        self.ax2 = Axes3D(self.figure2)
    
    def click(self):
        pass
    
    def CreateWindowGraph(self):
        pass
    
    def PrintFormula(self, args):
        if len(args) < 6:
            try:
                x = np.arange(float(args[0]),float(args[1]),float(args[2]))
            except ValueError:
                QMessageBox.critical(self, "Ошибка ", "Неправильные параметры 'x'", QMessageBox.Ok)
                return
            except ZeroDivisionError:
                QMessageBox.critical(self, "Ошибка ", "Неподходящий диапазон 'x'", QMessageBox.Ok)
                return
            except SyntaxError:
                QMessageBox.critical(self, "Ошибка ", "Неверная формула", QMessageBox.Ok)
                return
        
            self.SetWidgetTrue()
        
            y = ne.evaluate(args[3])
        
            self.ax.grid(True)
            self.ax.plot(x,y)
            self.x, self.y = x,y
        else:
            try:
                x = np.arange(float(args[0]),float(args[1]),
                             np.abs((float(args[0])-float(args[1]))/float(args[4])))
                y = np.arange(float(args[2]),float(args[3]),
                             np.abs((float(args[2])-float(args[3]))/float(args[4])))
            except ValueError:
                QMessageBox.critical(self, "Ошибка ", "Неправильные параметры 'x'", QMessageBox.Ok)
                return
            except ZeroDivisionError:
                QMessageBox.critical(self, "Ошибка ", "Неподходящий диапазон 'x'", QMessageBox.Ok)
                return
            except SyntaxError:
                QMessageBox.critical(self, "Ошибка ", "Неверная формула", QMessageBox.Ok)
                return
            
            self.SetWidgetTrue()
            if self.cb_graph.isChecked():
                self.w2.setVisible(True)
                x, y = np.meshgrid(x, y)
                self.surf = self.ax2.plot_surface(x, y, ne.evaluate(args[5]), cmap=plt.cm.coolwarm,
                                                  linewidth=0, antialiased=False)
            else:
                self.w1.setVisible(True)
                x, y = np.meshgrid(x, y)
                self.ax.contourf(x, y, ne.evaluate(args[5]))
            
    
    def PrintInput(self,args):
        pass
    
    def PrintFile(self, file):
        pass
    
    def Reset(self):
        qApp.exit( Window.EXIT_CODE_REBOOT )

#### ГЛАВНЫЙ ИНТЕРФЕЙС

In [12]:
class MainWindow(Window):
    def __init__(self):
        Window.__init__(self,ui = 'mainwindow.ui')
        
        self.inputline_2.setVisible(False)
        self.lineEdit_2.setVisible(False)
        
    def click(self):
        self.exit.clicked.connect(lambda: self.close())
        self.cb_i.stateChanged.connect(lambda: self.ChangeInput())
        self.cb_w.stateChanged.connect(lambda: self.ChangeInputWay())
        self.cb_f.stateChanged.connect(lambda: self.ChangeInputForm())
        
        #self.b_d.clicked.connect(lambda: print(1))
        self.b_f.clicked.connect(lambda: self.CreateWindowGraph())
        self.b_e.clicked.connect(lambda: self.CreateExtremGraph())
        
        self.reset.clicked.connect(lambda: self.Reset())
        
    def CheckBox(self):
        if self.cb_w.isChecked() == True and self.cb_i.isChecked() == True:
            QMessageBox.critical(self, "Неправильный ввод", "У вас выбрано 2 или более 'checkbox'", QMessageBox.Ok)
            return False
        elif self.cb_f.isChecked() == True and self.cb_i.isChecked() == True:
            QMessageBox.critical(self, "Неправильный ввод", "У вас выбрано 2 или более 'checkbox'", QMessageBox.Ok)
            return False
        elif self.cb_f.isChecked() == True and self.cb_w.isChecked() == True:
            QMessageBox.critical(self, "Неправильный ввод", "У вас выбрано 2 или более 'checkbox'", QMessageBox.Ok)
            return False
        else: 
            return True
    
    def ChangeInputForm(self):
        if self.CheckBox() == True:
            pass
        else: 
            self.cb_f.setChecked(False)
            
    def ChangeInputWay(self):
        if self.CheckBox() == True:
            if self.cb_w.isChecked() == True:
                self.lineEdit.setText('Путь : ')
            else:
                self.lineEdit.setText('f(x) = ')
        else: 
            self.cb_w.setChecked(False)
        
    def ChangeInput(self):
        if self.CheckBox() == True:
            if self.cb_i.isChecked() == True:
                self.inputline_2.setVisible(True)
                self.lineEdit_2.setVisible(True)
                self.lineEdit.setText('Y = ')
            else: 
                self.inputline_2.setVisible(False)
                self.lineEdit_2.setVisible(False)
                self.lineEdit.setText('f(x) = ')
        else:
            self.cb_i.setChecked(False)
        
    def CreateExtremGraph(self):
        if self.CheckBox() == True:
            if self.cb_f.isChecked() == True:
                self.w = WindowExtremum('formula',self.inputline.text())
                self.w.show()
            elif self.cb_w.isChecked() == True:
                self.w = WindowExtremum(['file',self.inputline.text()])
                self.w.show()
        
    def CreateWindowGraph(self):
        if self.CheckBox() == True:
            if self.cb_f.isChecked() == True:
                self.w = WindowGraph(['formula',self.inputline.text()])
                self.w.show()
            elif self.cb_i.isChecked() == True:
                self.w = WindowGraph(['input',[self.inputline_2.text(),self.inputline.text()]])
                self.w.show()
            elif self.cb_w.isChecked() == True:
                self.w = WindowGraph(['file',self.inputline.text()])
                self.w.show()

#### ИНТЕРФЕЙС ГРАФИКОВ(с формулы/входных данных/с файла)

In [13]:
class WindowGraph(Window):
    def __init__(self, argv):
        inputType = argv[0]
        Window.__init__(self,'graph.ui')
        self.SetWidget()
        
        if (inputType == 'formula'):
            formula = argv[1]
            self.formula.setText(formula)
            self.SetWidgetFalse()
            
        elif (inputType == 'input'):
            args = argv[1]
            ar = []
            s = args[0].split()
            num_list = []
 
            for word in s:
                num_list.append(ne.evaluate(word))
            ar.append(num_list)
            
            s = args[1].split()
            num_list = []
            for word in s:
                num_list.append(ne.evaluate(word))
            ar.append(num_list)
            
            self.PrintInput(ar)
        
        elif (inputType == 'file'):
            self.formula.setText(argv[1])
            self.PrintFile(argv[1])
        
        
    def SetWidgetFalse(self):
        self.save_res.setVisible(False)
        self.w1.setVisible(False)
        self.back.setVisible(False)
        
        
        self.minL.setVisible(True)
        self.maxL.setVisible(True)
        self.step.setVisible(True)
        self.show_g.setVisible(True)
        
        self.minX.setVisible(True)
        self.maxX.setVisible(True)
        self.stepT.setVisible(True)
        
        self.ax.clear()
    
    def SetWidgetTrue(self):
        self.minL.setVisible(False)
        self.maxL.setVisible(False)
        self.step.setVisible(False)
        self.show_g.setVisible(False)
        
        self.minX.setVisible(False)
        self.maxX.setVisible(False)
        self.stepT.setVisible(False)
        
        self.save_res.setVisible(True)
        self.w1.setVisible(True)
        self.back.setVisible(True)
        
    def click(self):
        self.show_g.clicked.connect(lambda: self.PrintFormula([self.minL.text(),self.maxL.text(),self.step.text(),self.formula.text()]))
        self.back.clicked.connect(lambda: self.SetWidgetFalse())
        
    def PrintFile(self, file):
        self.SetWidgetTrue()
        self.back.setVisible(False)
        
    def PrintInput(self,args):
        self.x, self.y = args[0],args[1]
        self.SetWidgetTrue()
        self.back.setVisible(False)
        self.lineEdit_3.setVisible(False)
        self.formula.setVisible(False)
        
        self.ax.grid(True)
        try:
            self.ax.plot(self.x, self.y)
        except ValueError:
            QMessageBox.critical(self, "Ошибка ", "Несовпадение размеров 'x' и 'y'", QMessageBox.Ok)
            self.deleteLater()
            return

#### ИНТЕРФФЕЙС РАСПРЕДЕЛЕНИЙ

In [14]:
class Distribution(Window):
    def __init__(self):
        Window.__init__(self,ui = 'distribution.ui')

#### РАСЧЁТ ЭКСТРЕМУМА

--------------------------------------
ТУТ НАДА НАПИСАТЬ ЧТО-ТО
--------------------------------------
--------------------------------------

#### ИНТЕРФФЕЙС ЭКСТРЕМУМА

In [15]:
class WindowExtremum(Window):
    def __init__(self,typeC,arg):
        Window.__init__(self,ui = 'extremum.ui')
        self.SetWidget()
        self.SetWidget3D()
        if typeC == 'formula':
            self.formula.setText(arg)
            self.SetWidgetFalse()
        elif typeC == 'file':
            self.PrintFile(arg)
        
    def SetWidgetFalse(self):
        self.w1.setVisible(False)
        self.w2.setVisible(False)
        self.ax2.clear()
        self.ax.clear()
        
        self.cb_graph.setVisible(True)
        
        self.label.setVisible(True)
        self.label_2.setVisible(True)
        self.label_3.setVisible(True)
        self.label_4.setVisible(True)
        self.label_5.setVisible(True)
        self.label_6.setVisible(True)
        self.label_7.setVisible(True)
        self.calcul.setVisible(True)
        self.lineEdit.setVisible(True)
        self.lineEdit_2.setVisible(True)
        self.lineEdit_4.setVisible(True)
        self.lineEdit_5.setVisible(True)
        self.lineEdit_6.setVisible(True)
        self.lineEdit_7.setVisible(True)
        self.l1.setVisible(True)
        self.l2.setVisible(True)
        self.l3.setVisible(True)
        self.l4.setVisible(True)
        self.l5.setVisible(True)
        self.l6.setVisible(True)
        self.l7.setVisible(True)
    
    def SetWidgetTrue(self):
        self.cb_graph.setVisible(False)
        
        self.label.setVisible(False)
        self.label_2.setVisible(False)
        self.label_3.setVisible(False)
        self.label_4.setVisible(False)
        self.label_5.setVisible(False)
        self.label_6.setVisible(False)
        self.label_7.setVisible(False)
        self.calcul.setVisible(False)
        self.lineEdit.setVisible(False)
        self.lineEdit_2.setVisible(False)
        self.lineEdit_4.setVisible(False)
        self.lineEdit_5.setVisible(False)
        self.lineEdit_6.setVisible(False)
        self.lineEdit_7.setVisible(False)
        self.l1.setVisible(False)
        self.l2.setVisible(False)
        self.l3.setVisible(False)
        self.l4.setVisible(False)
        self.l5.setVisible(False)
        self.l6.setVisible(False)
        self.l7.setVisible(False)
    
    def click(self):
        self.back.clicked.connect(lambda: self.SetWidgetFalse())
        self.calcul.clicked.connect(lambda: self.PrintFormula([self.l1.text(),self.l2.text(),self.l3.text(),self.l4.text(),
                                                              self.l5.text(),self.formula.text()]))
        
        self.cb_graph.stateChanged.connect(lambda: self.ChangeOutput())
    
    def ChangeOutput(self):
        if self.cb_graph.isChecked():
            self.cb_graph.setText('3д график')
        else:
            self.cb_graph.setText('Градиент')
    
    #sin(sqrt(x*x+y*y))

In [16]:
def main():
    currentExitCode = Window.EXIT_CODE_REBOOT
    while currentExitCode == MainWindow.EXIT_CODE_REBOOT:
        app = QApplication(sys.argv)
        window = MainWindow()
        #window = WindowGraph()
        window.show()
        currentExitCode = app.exec_()
        app = None

In [ ]:
if __name__ == '__main__':
     main()

In [ ]:
#https://pypi.org/project/numexpr/2.6.1/